In [1]:
# VADER: https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-rule-based-vader-and-nltk-72067970fb71

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.sparse

In [2]:
data = pd.read_pickle("./data/cleaned_submissions.pkl")
data = data.loc[data["sentiment"] != "", ["text", "sentiment"]] # Only get labeled instances

In [3]:
# Encode labels
lenc = LabelEncoder()
y = lenc.fit_transform(data["sentiment"])

# Vectorize text using tfidf
tfidf = TfidfVectorizer(preprocessor=' '.join, lowercase=False, min_df=5) # min_df = Minimum occurance of words
X = tfidf.fit_transform(data["text"])

In [4]:
X = scipy.sparse.load_npz('./data/X_sparse.npz')
y = np.load("./data/y_sparse.npy")

In [5]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# SVM

In [14]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

pipeline = Pipeline([
        ("svc", SVC())
    ])

param_grid = {"svc__kernel": ["poly"], "svc__C": [3]} #["poly", "rbf", "sigmoid", "linear"], [3, 4, 5, 6, 7]
CV = GridSearchCV(pipeline, param_grid, cv = 5)
# pipeline.get_params().keys() See all available parameters
CV.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('svc', SVC())]),
             param_grid={'svc__C': [3], 'svc__kernel': ['poly']})

In [17]:
import pickle
pickle.dump(CV, open("./data/svm_first_model_poly_3.sav", "wb"))

In [18]:
loaded_model = pickle.load(open("./data/svm_first_model_poly_3.sav", "rb"))

In [19]:
print("Accuracy of {} can be achieved with the following parameters: {}".format(loaded_model.score(X_test, y_test), CV.best_params_))

Accuracy of 0.8837895792141246 can be achieved with the following parameters: {'svc__C': 3, 'svc__kernel': 'poly'}


# Naive Bayes

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

pipeline = Pipeline([
        ("mnb", MultinomialNB())
    ])

param_grid = {"mnb__alpha": [1]}
CV = GridSearchCV(pipeline, param_grid, cv = 5)
# pipeline.get_params().keys() See all available parameters
CV.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('mnb', MultinomialNB())]),
             param_grid={'mnb__alpha': [1]})

In [23]:
import pickle
pickle.dump(CV, open("./data/nb_first_model_mnb.sav", "wb"))

In [24]:
loaded_model = pickle.load(open("./data/nb_first_model_mnb.sav", "rb"))

In [25]:
print("Accuracy of {} can be achieved with the following parameters: {}".format(loaded_model.score(X_test, y_test), CV.best_params_))

Accuracy of 0.7632626918042831 can be achieved with the following parameters: {'mnb__alpha': 1}


# LSTM

In [6]:
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [7]:
# DEV!!!
# Less data, so I can check if the model works
X_train_lstm = X_train.toarray()[:, :, None]
y_train_lstm = y_train[:]

In [8]:
X_train.shape

(143635, 19267)

In [9]:
X_train_lstm.shape

(143635, 19267, 1)

In [29]:
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
print(tf.config.list_physical_devices())

2.7.0
Num GPUs Available:  0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [28]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13416955267035498473
xla_global_id: -1
]


In [26]:
from keras.models import Sequential
from keras.layers import Embedding, Dense

def build_model(dims, hidden_states, opt):
    model = Sequential()
    model.add(Embedding(19267, dim))
    model.add(LSTM(hidden_states))
    model.add(Dense(1, activation = "softmax"))
    model.compile(opt, "categorical_crossentropy", metrics = ["acc"])

In [30]:
model = Sequential()
model.add(Embedding(19267, 256))
model.add(LSTM(16))
model.add(Dense(1, activation = "softmax"))
model.compile("rmsprop", "categorical_crossentropy", metrics = ["acc"])
history = model.fit(X_train_lstm, y_train_lstm,
                        epochs=2,
                        batch_size=32,
                        verbose=0)

In [54]:
# Save Model
from keras.models import load_model
history.model.save('./data/small_lstm_model.h5')

In [55]:
loaded_model = load_model('./data/small_lstm_model.h5')

In [57]:
print(loaded_model.history)

None


In [62]:
print(loaded_model.history)

None


In [42]:
print(history.history) # Loss and Accuracy

{'loss': [0.0, 0.0], 'acc': [0.6518188714981079, 0.6518188714981079]}
{'verbose': 0, 'epochs': 2, 'steps': 4489}


In [64]:
import pickle
with open('./data/train_history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [65]:
with open('./data/train_history', 'rb') as file_pi:
    h = pickle.load(file_pi)

In [66]:
print(h)

{'loss': [0.0, 0.0], 'acc': [0.6518188714981079, 0.6518188714981079]}


In [27]:
dim = [256] #[256, 512, 1024]
hidden_states = [16] #[16, 32, 64]
optimi = ["rmsprop"] #["rmsprop", "SGD", "Adam"]

acc = []
val_acc = []
loss = []
val_loss = []

param_list = []

for d in dim:
    for state in hidden_states:
        for opt in optimi:
            history = model.fit(X_train, y_train,
                        epochs=2,
                        batch_size=32,
                        validation_split=0.2,
                        verbose=0)
            acc.append(history.history['acc'])
            val_acc.append(history.history['val_acc'])
            loss.append(history.history['loss'])
            val_loss.append(history.history['val_loss'])
            param_list.append("Optimizer: " + opt + " - States: " + str(state) + " - Dimensions:" + str(d))


NameError: name 'model' is not defined

In [32]:
import matplotlib.pyplot as plt

def plot_history(data_list, label_list, title, xlabel='Epochs', ylabel=None):
    ''' Plots a list of vectors.

    Parameters:
        data_list  : list of vectors containing the values to plot
        label_list : list of labels describing the data, one per vector
        title      : title of the plot
        ylabel     : label for the y axis
    '''
    epochs = range(1, len(data_list[0]) + 1)

    for data, label in zip(data_list, label_list):
        plt.plot(epochs, data, label=label)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()

    plt.show()


In [33]:
plot_history(data_list=val_loss,
             label_list=[[""]*len(param_list)], #param_list,
             title='Comparison of different recurrent layer types',
             ylabel='Loss')
plot_history(data_list=val_acc,
             label_list=[[""]*len(param_list)], #param_list,
             title='Comparison of different recurrent layer types',
             ylabel='Validation accuracy')

NameError: name 'val_loss' is not defined

# BERT